Bootcamp: Arquiteto(a) de Big Data
Trabalho Prático
Módulo 4 Processamento de dados utilizando o ecossistema Hadoop

Baixe o dataset airports.dat do seguinte link: https://openflights.org/data.html

github.com/odiego

08/10/2022

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 12.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8fb5b988235cadb1f10552db1f5493f5bb21a9d9456cca94fa0cb33cdc02205e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


1) Iniciar a seção SPARK

In [2]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [3]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("desafio_IGTI").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [4]:
diretorio_dataset='/content/airports.csv'#diretório que contém o arquivo a ser utilizado

In [5]:
dataset_desafio = spark.read.format("csv").options(header="false", sep =',', inferschema="true").load(diretorio_dataset)  #realiza a leitura do dataset

In [6]:
dataset_desafio.printSchema() #mostra o esquema inferido pelas variáveis

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)



Respectivamente, quantas variáveis do tipo string e inteira existem no dataset?

In [7]:
#número de instancias no dataset
dataset_desafio.count()

7698

Quantas instâncias e atributos, respectivamente, existem no dataset?

In [8]:
dataset_desafio.show() #mostra as linhas iniciais do dataset

+---+--------------------+--------------+----------------+---+----+------------------+-------------------+----+---+----+--------------------+-------+-----------+
|_c0|                 _c1|           _c2|             _c3|_c4| _c5|               _c6|                _c7| _c8|_c9|_c10|                _c11|   _c12|       _c13|
+---+--------------------+--------------+----------------+---+----+------------------+-------------------+----+---+----+--------------------+-------+-----------+
|  1|      Goroka Airport|        Goroka|Papua New Guinea|GKA|AYGA|-6.081689834590001|      145.391998291|5282| 10|   U|Pacific/Port_Moresby|airport|OurAirports|
|  2|      Madang Airport|        Madang|Papua New Guinea|MAG|AYMD|    -5.20707988739|      145.789001465|  20| 10|   U|Pacific/Port_Moresby|airport|OurAirports|
|  3|Mount Hagen Kagam...|   Mount Hagen|Papua New Guinea|HGU|AYMH|-5.826789855957031| 144.29600524902344|5388| 10|   U|Pacific/Port_Moresby|airport|OurAirports|
|  4|      Nadzab Airport|  

Renomeando as colunas...

In [14]:
df2 = dataset_desafio.withColumnRenamed("_c0","id") \
  .withColumnRenamed("_c1","name") \
  .withColumnRenamed("_c2","city") \
  .withColumnRenamed("_c3","country") \
  .withColumnRenamed("_c4","iata") \
  .withColumnRenamed("_c5","icao") \
           .withColumnRenamed("_c6","lat") \
           .withColumnRenamed("_c7","long") \
           .withColumnRenamed("_c8","altitude") \
           .withColumnRenamed("_c9","timezone") \
           .withColumnRenamed("_c10","dst") \
           .withColumnRenamed("_c11","timezone") \
           .withColumnRenamed("_c12","type") \
           .withColumnRenamed("_c13","source")
df2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- iata: string (nullable = true)
 |-- icao: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- altitude: integer (nullable = true)
 |-- timezone: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source: string (nullable = true)



In [39]:
df2.show() # Exibindo o dataset final que será trabalhado

+---+--------------------+--------------+----------------+----+----+------------------+-------------------+--------+--------+---+--------------------+-------+-----------+
| id|                name|          city|         country|iata|icao|               lat|               long|altitude|timezone|dst|            timezone|   type|     source|
+---+--------------------+--------------+----------------+----+----+------------------+-------------------+--------+--------+---+--------------------+-------+-----------+
|  1|      Goroka Airport|        Goroka|Papua New Guinea| GKA|AYGA|-6.081689834590001|      145.391998291|    5282|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|  2|      Madang Airport|        Madang|Papua New Guinea| MAG|AYMD|    -5.20707988739|      145.789001465|      20|      10|  U|Pacific/Port_Moresby|airport|OurAirports|
|  3|Mount Hagen Kagam...|   Mount Hagen|Papua New Guinea| HGU|AYMH|-5.826789855957031| 144.29600524902344|    5388|      10|  U|Pacific/Port_Mor

Considerando o dataset airports.dat, quantos aeroportos o Brasil (está em inglês, Brazil) possui?

Considerando o dataset airports.dat, qual país apresenta a maior quantidade de aeroportos?

In [18]:
df2.groupby('country').count().sort("count",ascending=False).show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 1512|
|        Canada|  430|
|     Australia|  334|
|        Russia|  264|
|        Brazil|  264|
|       Germany|  249|
|         China|  241|
|        France|  217|
|United Kingdom|  167|
|         India|  148|
|     Indonesia|  145|
|         Japan|  123|
|  South Africa|   99|
|     Argentina|   96|
|        Mexico|   84|
|         Italy|   83|
|          Iran|   82|
|        Sweden|   77|
|        Turkey|   76|
|      Colombia|   75|
+--------------+-----+
only showing top 20 rows



Considerando o dataset airports.dat, qual a diferença de altitude entre os aeroportos Jorge Newbery Airpark e Altamira Airport?

In [63]:
df2.filter(df2.name == 'Altamira Airport').show()

+----+----------------+--------+-------+----+----+----------------+----------------+--------+--------+---+-------------+-------+-----------+
|  id|            name|    city|country|iata|icao|             lat|            long|altitude|timezone|dst|     timezone|   type|     source|
+----+----------------+--------+-------+----+----+----------------+----------------+--------+--------+---+-------------+-------+-----------+
|2566|Altamira Airport|Altamira| Brazil| ATM|SBHT|-3.2539100646973|-52.254001617432|     369|      -3|  S|America/Belem|airport|OurAirports|
+----+----------------+--------+-------+----+----+----------------+----------------+--------+--------+---+-------------+-------+-----------+



In [64]:
df2.filter(df2.name == 'Jorge Newbery Airpark').show()

+----+--------------------+------------+---------+----+----+--------+--------+--------+--------+---+--------------------+-------+-----------+
|  id|                name|        city|  country|iata|icao|     lat|    long|altitude|timezone|dst|            timezone|   type|     source|
+----+--------------------+------------+---------+----+----+--------+--------+--------+--------+---+--------------------+-------+-----------+
|2442|Jorge Newbery Air...|Buenos Aires|Argentina| AEP|SABE|-34.5592|-58.4156|      18|      -3|  N|America/Buenos_Aires|airport|OurAirports|
+----+--------------------+------------+---------+----+----+--------+--------+--------+--------+---+--------------------+-------+-----------+



Considerando o dataset airports.dat, quantos aeroportos existem na cidade de São Paulo? (no dataset não há acentos)

In [19]:
df2.filter(df2.city == 'Sao Paulo').count()

3

Considerando o dataset airports.dat, qual cidade apresenta a maior quantidade de aeroportos?

In [20]:
df2.groupby('city').count().sort("count",ascending=False).show()

+------------+-----+
|        city|count|
+------------+-----+
|        null|   49|
|      London|    9|
|    Columbus|    8|
|  Georgetown|    7|
|    San Jose|    7|
|      Moscow|    6|
|  Greenville|    6|
|    Santiago|    6|
|    New York|    6|
|  Alexandria|    6|
|     Houston|    6|
|Jacksonville|    6|
| San Antonio|    5|
|  Burlington|    5|
|     Glasgow|    5|
|      Athens|    5|
|    Hamilton|    5|
|    Victoria|    5|
|   Vancouver|    5|
|        Rome|    5|
+------------+-----+
only showing top 20 rows



Considerando o dataset airports.dat, qual aeroporto apresenta a maior altitude?

In [45]:
df2.sort(df2.altitude.desc()).show(truncate=False)

+-----+--------------------------------------+-----------+---------+----+----+-------------------+------------------+--------+--------+---+--------------+-------+-----------+
|id   |name                                  |city       |country  |iata|icao|lat                |long              |altitude|timezone|dst|timezone      |type   |source     |
+-----+--------------------------------------+-----------+---------+----+----+-------------------+------------------+--------+--------+---+--------------+-------+-----------+
|9310 |Daocheng Yading Airport               |Daocheng   |China    |DCY |ZUDC|29.323056          |100.053333        |14472   |8       |N  |Asia/Shanghai |airport|OurAirports|
|6396 |Qamdo Bangda Airport                  |Bangda     |China    |BPX |ZUBD|30.553600311279297 |97.1082992553711  |14219   |8       |U  |Asia/Shanghai |airport|OurAirports|
|8921 |Kangding Airport                      |Kangding   |China    |KGT |ZUKD|30.1575            |101.734722        |14042   

Considerando o dataset airports.dat, qual a média da altitude dos aeroportos do Brasil?

In [66]:
df2.filter(df2.country == 'Brazil').agg({'altitude': 'mean'}).show()

+-----------------+
|    avg(altitude)|
+-----------------+
|1126.219696969697|
+-----------------+



Considerando o dataset airports.dat, quantos DST (faixas de horário de verão) existem? (Para essa resposta desconsidere aqueles que não possuem dados, representados por \N)

In [75]:
#df2.select('dst').distinct().show() # Listagem de DST unicos (incluindo o \N)
df2.select('dst').filter("dst <> '\\N'").distinct().count() # Contagem de unicos, excluindo o \N

7

Considerando o dataset airports.dat, quantos países possuem aeroportos?

In [48]:
df2.select('country').distinct().count()

237

In [47]:
df2.groupby('country').count().sort("count",ascending=True).show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|              Tuvalu|    1|
|      Midway Islands|    1|
|          Martinique|    1|
|British Indian Oc...|    1|
|         Isle of Man|    1|
|      Norfolk Island|    1|
|            Barbados|    1|
|             Grenada|    1|
|    Christmas Island|    1|
|          Montserrat|    1|
|Cocos (Keeling) I...|    1|
|           West Bank|    1|
|               Nauru|    1|
|            Anguilla|    1|
|           Gibraltar|    1|
|        Saint Helena|    1|
|           Palestine|    1|
|               Macau|    1|
|               Aruba|    1|
|              Jersey|    1|
+--------------------+-----+
only showing top 20 rows

